# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [1]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [3]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [4]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

  0%|          | 0/54288 [00:00<?, ?it/s]

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [7]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [8]:
your_name = "Vidushi" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [9]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [10]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
805,45,female,37.60575,0,no,7731.85785
1173,38,male,30.72300,2,no,6457.84340
0,19,female,29.29500,0,yes,16884.92400
681,19,male,21.31500,0,no,1242.26000
215,41,female,38.95500,2,no,7371.77200


Let us answer some basic questions about the dataset. 


**Q1: How many rows does the dataset have?**

In [11]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q2: How many columns doe the dataset have**

In [12]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q3: What are the column titles of the input variables?**

In [43]:
input_cols = ["age","sex","bmi","children","smoker"]
input_cols

['age', 'sex', 'bmi', 'children', 'smoker']

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [44]:
categorical_cols =["sex","smoker"]
categorical_cols

['sex', 'smoker']

**Q5: What are the column titles of output/target variable(s)?**

In [45]:
output_cols = ["charges"]
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

(charges    1121.8739
 dtype: float64,
 charges    63770.42801
 dtype: float64,
 charges    13276.262433
 dtype: float64,
 array([[<AxesSubplot:title={'center':'charges'}>]], dtype=object))

In [48]:
# Write your answer here
output=dataframe[output_cols]
output.min(),output.max(),output.mean(),output.hist("charges")

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [49]:
!pip install jovian --upgrade -q

In [50]:
import jovian

In [51]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


'https://jovian.com/vidushi-omar123/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [52]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [53]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[45.     ,  0.     , 37.60575,  0.     ,  0.     ],
        [38.     ,  1.     , 30.723  ,  2.     ,  0.     ],
        [19.     ,  0.     , 29.295  ,  0.     ,  1.     ],
        ...,
        [33.     ,  0.     , 35.175  ,  0.     ,  1.     ],
        [53.     ,  1.     , 30.324  ,  0.     ,  0.     ],
        [50.     ,  0.     , 48.3945 ,  1.     ,  0.     ]]),
 array([[ 7731.85785],
        [ 6457.8434 ],
        [16884.924  ],
        ...,
        [37079.372  ],
        [ 9869.8102 ],
        [ 9549.5651 ]]))

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [54]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)

In [55]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [56]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [57]:
val_percent = 0.14 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = torch.utils.data.random_split(dataset,[train_size,val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [58]:
batch_size = 100

In [59]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [60]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[45.0000,  1.0000, 24.0397,  2.0000,  1.0000],
        [22.0000,  1.0000, 33.7155,  0.0000,  0.0000],
        [45.0000,  0.0000, 29.2215,  2.0000,  0.0000],
        [64.0000,  0.0000, 41.6850,  0.0000,  0.0000],
        [63.0000,  0.0000, 26.3340,  0.0000,  0.0000],
        [19.0000,  0.0000, 32.1195,  2.0000,  0.0000],
        [54.0000,  0.0000, 22.5435,  3.0000,  0.0000],
        [39.0000,  1.0000, 27.5310,  1.0000,  0.0000],
        [30.0000,  0.0000, 34.9965,  1.0000,  0.0000],
        [47.0000,  1.0000, 26.6805,  1.0000,  1.0000],
        [40.0000,  1.0000, 43.2915,  1.0000,  0.0000],
        [58.0000,  0.0000, 33.4162,  2.0000,  0.0000],
        [28.0000,  1.0000, 39.9630,  0.0000,  0.0000],
        [23.0000,  0.0000, 34.4190,  2.0000,  1.0000],
        [47.0000,  0.0000, 25.3050,  1.0000,  0.0000],
        [61.0000,  1.0000, 35.2118,  0.0000,  0.0000],
        [51.0000,  1.0000, 31.5315,  1.0000,  0.0000],
        [21.0000,  1.0000, 32.8177,  0.0000,  0.0000],
  

Let's save our work by committing to Jovian.

In [61]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


'https://jovian.com/vidushi-omar123/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [62]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [63]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size,output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.mse_loss(out,targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out,targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [64]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [65]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2891,  0.2858,  0.2428, -0.0413, -0.4270]], requires_grad=True),
 Parameter containing:
 tensor([-0.2105], requires_grad=True)]

One final commit before we train the model.

In [66]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


'https://jovian.com/vidushi-omar123/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [67]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
       # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [68]:
result=evaluate(model,val_loader) # Use the the evaluate function
print(result)

{'val_loss': 341789184.0}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [72]:
epochs = 1000
lr =1e-6
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 82892992.0000
Epoch [40], val_loss: 82923504.0000
Epoch [60], val_loss: 82930416.0000
Epoch [80], val_loss: 82929464.0000
Epoch [100], val_loss: 82924424.0000
Epoch [120], val_loss: 82918336.0000
Epoch [140], val_loss: 82911552.0000
Epoch [160], val_loss: 82907192.0000
Epoch [180], val_loss: 82902296.0000
Epoch [200], val_loss: 82894200.0000
Epoch [220], val_loss: 82889632.0000
Epoch [240], val_loss: 82883208.0000
Epoch [260], val_loss: 82878808.0000
Epoch [280], val_loss: 82872288.0000
Epoch [300], val_loss: 82865304.0000
Epoch [320], val_loss: 82859976.0000
Epoch [340], val_loss: 82855512.0000
Epoch [360], val_loss: 82847464.0000
Epoch [380], val_loss: 82842552.0000
Epoch [400], val_loss: 82835824.0000
Epoch [420], val_loss: 82829968.0000
Epoch [440], val_loss: 82824376.0000
Epoch [460], val_loss: 82817432.0000
Epoch [480], val_loss: 82811936.0000
Epoch [500], val_loss: 82807128.0000
Epoch [520], val_loss: 82799784.0000
Epoch [540], val_loss: 82794224.0000
Epoch

In [73]:
epochs = 1000
lr = 1e-5
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 82630064.0000
Epoch [40], val_loss: 82594264.0000
Epoch [60], val_loss: 82524328.0000
Epoch [80], val_loss: 82443072.0000
Epoch [100], val_loss: 82311232.0000
Epoch [120], val_loss: 82341800.0000
Epoch [140], val_loss: 82244112.0000
Epoch [160], val_loss: 82221088.0000
Epoch [180], val_loss: 82104280.0000
Epoch [200], val_loss: 82055168.0000
Epoch [220], val_loss: 82016336.0000
Epoch [240], val_loss: 81926008.0000
Epoch [260], val_loss: 81880232.0000
Epoch [280], val_loss: 81802640.0000
Epoch [300], val_loss: 81812320.0000
Epoch [320], val_loss: 81702256.0000
Epoch [340], val_loss: 81628864.0000
Epoch [360], val_loss: 81544904.0000
Epoch [380], val_loss: 81521640.0000
Epoch [400], val_loss: 81496376.0000
Epoch [420], val_loss: 81406912.0000
Epoch [440], val_loss: 81347432.0000
Epoch [460], val_loss: 81336544.0000
Epoch [480], val_loss: 81228176.0000
Epoch [500], val_loss: 81149880.0000
Epoch [520], val_loss: 81134544.0000
Epoch [540], val_loss: 81059904.0000
Epoch

In [74]:
epochs = 2000
lr = 1e-5
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 79645008.0000
Epoch [40], val_loss: 79628720.0000
Epoch [60], val_loss: 79629648.0000
Epoch [80], val_loss: 79517384.0000
Epoch [100], val_loss: 79468448.0000
Epoch [120], val_loss: 79391360.0000
Epoch [140], val_loss: 79372784.0000
Epoch [160], val_loss: 79287552.0000
Epoch [180], val_loss: 79213848.0000
Epoch [200], val_loss: 79156040.0000
Epoch [220], val_loss: 79143600.0000
Epoch [240], val_loss: 79082352.0000
Epoch [260], val_loss: 79024944.0000
Epoch [280], val_loss: 78963832.0000
Epoch [300], val_loss: 78896688.0000
Epoch [320], val_loss: 78851616.0000
Epoch [340], val_loss: 78794400.0000
Epoch [360], val_loss: 78790264.0000
Epoch [380], val_loss: 78673448.0000
Epoch [400], val_loss: 78595648.0000
Epoch [420], val_loss: 78564112.0000
Epoch [440], val_loss: 78500504.0000
Epoch [460], val_loss: 78462520.0000
Epoch [480], val_loss: 78405568.0000
Epoch [500], val_loss: 78360576.0000
Epoch [520], val_loss: 78289656.0000
Epoch [540], val_loss: 78260928.0000
Epoch

In [75]:
epochs = 2000
lr =1e-4
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 74228728.0000
Epoch [40], val_loss: 73489544.0000
Epoch [60], val_loss: 73769512.0000
Epoch [80], val_loss: 72443488.0000
Epoch [100], val_loss: 72582608.0000
Epoch [120], val_loss: 71551688.0000
Epoch [140], val_loss: 71272256.0000
Epoch [160], val_loss: 70659936.0000
Epoch [180], val_loss: 70232096.0000
Epoch [200], val_loss: 71432136.0000
Epoch [220], val_loss: 69948992.0000
Epoch [240], val_loss: 68966984.0000
Epoch [260], val_loss: 68717648.0000
Epoch [280], val_loss: 68641176.0000
Epoch [300], val_loss: 67981184.0000
Epoch [320], val_loss: 67709040.0000
Epoch [340], val_loss: 67021288.0000
Epoch [360], val_loss: 66676624.0000
Epoch [380], val_loss: 66299560.0000
Epoch [400], val_loss: 66137940.0000
Epoch [420], val_loss: 65566152.0000
Epoch [440], val_loss: 65228456.0000
Epoch [460], val_loss: 65679544.0000
Epoch [480], val_loss: 66605952.0000
Epoch [500], val_loss: 64130136.0000
Epoch [520], val_loss: 64034944.0000
Epoch [540], val_loss: 63874216.0000
Epoch

**Q12: What is the final validation loss of your model?**

In [84]:
val_loss = history4[-1]
val_loss

{'val_loss': 48625648.0}

Let's log the final validation loss to Jovian and commit the notebook

In [85]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [86]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


'https://jovian.com/vidushi-omar123/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [87]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)              # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [88]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([41.0000,  0.0000, 33.2168,  1.0000,  0.0000])
Target: tensor([7358.1758])
Prediction: tensor([nan])


In [89]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([49.0000,  1.0000, 27.1320,  2.0000,  1.0000])
Target: tensor([23807.2402])
Prediction: tensor([nan])


In [90]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([55.0000,  0.0000, 28.3290,  0.0000,  0.0000])
Target: tensor([11082.5771])
Prediction: tensor([nan])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [91]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


<IPython.core.display.Javascript object>

[jovian] Updating notebook "vidushi-omar123/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/vidushi-omar123/02-insurance-linear-regression


'https://jovian.com/vidushi-omar123/02-insurance-linear-regression'